This notebook covers a Python-based solution for the fourth programming exercise of the machine learning class on Coursera.  Please refer to the [exercise text](https://github.com/jdwittenauer/ipython-notebooks/blob/master/exercises/ML/ex4.pdf) for detailed descriptions and equations.

# Machine Learning Exercise 4 - Neural Networks

In this exercise, we'll be working with a dataset to predict student performance using a feed-forward neural network with backpropagation.  We'll implement un-regularized and regularized versions of the neural network cost function and gradient computation via the backpropagation algorithm.  We'll also implement random weight initialization and a method to use the network to make predictions.

Since the data set is the same one we used in exercise 3, we'll re-use the code to load the data.

Since we're going to need these later (and will use them often), let's create some useful variables up-front.

In [84]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from scipy.optimize import minimize

In [85]:
# Load the data
data = pd.read_csv('Student_Performance.csv')
label_encoder = LabelEncoder()

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours_Studied                     499 non-null    int64  
 1   Previous_Scores                   499 non-null    int64  
 2   Extracurricular_Activities        499 non-null    object 
 3   Sleep_Hours                       499 non-null    int64  
 4   Sample_Question_Papers_Practiced  499 non-null    int64  
 5   Performance_Index                 499 non-null    float64
dtypes: float64(1), int64(4), object(1)
memory usage: 23.5+ KB


In [86]:
data['Extracurricular_Activities'] = label_encoder.fit_transform(data['Extracurricular_Activities'])
# Define X and y
X = data.drop('Performance_Index', axis=1).values
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(data['Sample_Question_Papers_Practiced'])


X.shape, y.shape


((499, 5), (499, 10))

We're also going to need to one-hot encode our y labels.  One-hot encoding turns a class label n (out of k classes) into a vector of length k where index n is "hot" (1) while the rest are zero.  Scikit-learn has a built in utility we can use for this.

The neural network for this exercise will have an input layer sized according to the features in our dataset, a hidden layer with a customizable number of units, and an output layer designed for classification. For additional details and an image of the network architecture, please refer to the PDF in the "exercises" folder.

The first piece we need to implement is a cost function to evaluate the loss for a given set of network parameters.  The source mathematical function is in the exercise text (and looks pretty intimidating).  Here are the functions required to compute the cost.

In [87]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [88]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = np.dot(a1, theta1.T)
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = np.dot(a2, theta2.T)
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h



In [89]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    return J

We've used the sigmoid function before so that's not new.  The forward-propagate function computes the hypothesis for each training instance given the current parameters.  It's output shape should match the same of our one-hot encoding for y.  We can test this real quick to convince ourselves that it's working as expected (the intermediate steps are also returned as these will be useful later).

In [90]:
# initial setup
input_size = X.shape[1]  # This will now adjust based on your data
hidden_size = 25  # You can adjust this if needed
num_labels = y.shape[1]   # Assuming you have  classes
learning_rate = 0.1

# randomly initialize a parameter array of the size of the full network's parameters

params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25
m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((25, 6), (10, 26))

In [91]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((499, 6), (499, 25), (499, 26), (499, 10), (499, 10))

The cost function, after computing the hypothesis matrix h, applies the cost equation to compute the total error between y and h.

In [92]:
cost(params, input_size, hidden_size, num_labels, X, y, learning_rate)

6.5713211367990825

Our next step is to add regularization to the cost function.  If you're following along in the exercise text and thought the last equation looked ugly, this one looks REALLY ugly.  It's actually not as complicated as it looks though - in fact, the regularization term is simply an addition to the cost we already computed.  Here's the revised cost function.

In [93]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [94]:
cost(params, input_size, hidden_size, num_labels, X, y, learning_rate)

6.571509202982251

Next up is the backpropagation algorithm.  Backpropagation computes the parameter updates that will reduce the error of the network on the training data.  The first thing we need is a function that computes the gradient of the sigmoid function we created earlier.

In [95]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

Now we're ready to implement backpropagation to compute the gradients.  Since the computations required for backpropagation are a superset of those required in the cost function, we're actually going to extend the cost function to also perform backpropagation and return both the cost and the gradients.

In [96]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, num_labels)
        yt = y[t,:]  # (1, num_labels)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

The hardest part of the backprop computation (other than understanding WHY we're doing all these calculations) is getting the matrix dimensions right.  By the way, if you find it confusing when to use A * B vs. np.multiply(A, B), you're not alone.  Basically the former is a matrix multiplication and the latter is an element-wise multiplication (unless A or B is a scalar value, in which case it doesn't matter).  I wish there was a more concise syntax for this (maybe there is and I'm just not aware of it).

Anyway, let's test it out to make sure the function returns what we're expecting it to.

In [97]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate)
J, grad.shape

(6.571509202982251, (410,))

We still have one more modification to make to the backprop function - adding regularization to the gradient calculations.  The final regularized version is below.

In [98]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [99]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate)
J, grad.shape

(6.571509202982251, (410,))

We are now ready to train our neural network using the 'TNC' optimization method and then use it for making predictions.

This is roughly similar to the previous exercise with multi-class logistic regression.

In [100]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

C:\Users\L\AppData\Local\Temp\ipykernel_3592\3203356950.py:4: OptimizeWarning: Unknown solver options: maxiter
  fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y, learning_rate),
C:\Users\L\AppData\Local\Temp\ipykernel_3592\3867835023.py:21: RuntimeWarning: divide by zero encountered in log
  second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
C:\Users\L\AppData\Local\Temp\ipykernel_3592\3867835023.py:21: RuntimeWarning: invalid value encountered in multiply
  second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))


 message: Linear search failed
 success: False
  status: 4
     fun: 1.650865200263948
       x: [-6.490e-02 -6.492e-02 ... -3.860e-01 -3.898e-01]
     nit: 34
     jac: [-3.621e-06  1.212e-04 ... -6.180e-03 -6.181e-03]
    nfev: 351

We put a bound on the number of iterations since the objective function is not likely to completely converge.  Our total cost has dropped below 0.5 though so that's a good indicator that the algorithm is working.  Let's use the parameters it found and forward-propagate them through the network to get some predictions.

In [101]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

y_pred = np.array(np.argmax(h, axis=1))
y_pred


array([[1],
       [2],
       [2],
       [2],
       [6],
       [6],
       [6],
       [6],
       [2],
       [0],
       [5],
       [2],
       [2],
       [2],
       [9],
       [3],
       [4],
       [2],
       [9],
       [0],
       [3],
       [0],
       [6],
       [6],
       [3],
       [4],
       [9],
       [6],
       [5],
       [3],
       [3],
       [1],
       [9],
       [1],
       [3],
       [4],
       [3],
       [2],
       [1],
       [3],
       [4],
       [0],
       [0],
       [5],
       [4],
       [3],
       [0],
       [6],
       [3],
       [2],
       [6],
       [1],
       [9],
       [0],
       [3],
       [7],
       [5],
       [9],
       [0],
       [0],
       [4],
       [5],
       [6],
       [2],
       [5],
       [2],
       [8],
       [3],
       [1],
       [8],
       [9],
       [7],
       [5],
       [2],
       [1],
       [3],
       [3],
       [0],
       [9],
       [9],
       [8],
       [6],
       [5],
    

Finally we can compute the accuracy to see how well our trained network is doing.

In [104]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, np.argmax(y, axis=1))]
accuracy = sum(correct) / len(correct)
print('accuracy = {:.2f}%'.format(accuracy * 100))


accuracy = 84.77%


And we're done!  We've successfully implemented a rudimentary feed-forward neural network with backpropagation and used it to classify images of handwritten digits.  In the next exercise we'll look at another power supervised learning algorithm, support vector machines.